<center><h1>Hou_Shengtao_HW3</h1></center>
<br>
<br>

Name: Shengtao Hou
<br>
Github Username: Tonyhou521 
<br>
USC ID: 9712707229

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [1]:
import os
import numpy as np
import pandas as pd
## ignore warnings
import warnings
warnings.filterwarnings('ignore')
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats


Get the AReM Data Set

In [2]:
# import csv

# ssv_file_path = '../data/AReM/bending2/dataset4.csv'  # replace with the actual input file path
# csv_file_path = '../data/AReM/bending2/Ndataset4.csv'  # replace with the actual output file path

# with open(ssv_file_path, 'r') as ssv_file, open(csv_file_path, 'w', newline='') as csv_file:
#     ssv_reader = csv.reader(ssv_file, delimiter=' ')
#     csv_writer = csv.writer(csv_file, delimiter=',')

#     for row in ssv_reader:
#         csv_writer.writerow(row)

In [3]:
# ## go through each csv file and convert each to a dataframe and show info
# ## import to print all output
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
# dir_path = '../data/AReM/'
# for root, dirs, files in os.walk(dir_path):
#     for file in files:
#         if file.endswith('.csv'):
#             file_path = os.path.join(root, file)
#             print(file_path)
#             df = pd.read_csv(file_path, skiprows=4)
#             print()

### (b) Test and Train Data


In [4]:
## Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1,2, and 3 in other folders as test data and other datasets as train data.
dir_path = '../data/AReM/'

train_df = pd.DataFrame()
test_df = pd.DataFrame()

for root, dirs, files in os.walk(dir_path):
    for each in files:
        if each.endswith('.csv'):
            file_path = os.path.join(root, each)
            if 'bending1' in file_path or 'bending2' in file_path:
                if 'dataset1' in file_path or 'dataset2' in file_path:
                    test_df = test_df.append(pd.read_csv(file_path, skiprows=4))
                else:
                    train_df = train_df.append(pd.read_csv(file_path, skiprows=4))
            else:
                if 'dataset1' in file_path or 'dataset2' in file_path or 'dataset3' in file_path:
                    test_df = test_df.append(pd.read_csv(file_path, skiprows=4))
                else:
                    train_df = train_df.append(pd.read_csv(file_path, skiprows=4))


# change first column name to time
train_df.rename(columns={'# Columns: time': 'Time'}, inplace=True)
test_df.rename(columns={'# Columns: time': 'Time'}, inplace=True)

display(test_df)
display(train_df)




,Time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,39.25,0.43,22.75,0.43,33.75,1.30
1,250,39.25,0.43,23.00,0.00,33.00,0.00
2,500,39.25,0.43,23.25,0.43,33.00,0.00
3,750,39.50,0.50,23.00,0.71,33.00,0.00
4,1000,39.50,0.50,24.00,0.00,33.00,0.00
...,...,...,...,...,...,...,...
475,118750,35.00,3.00,21.50,2.50,15.75,7.76
476,119000,40.25,2.49,19.75,3.27,21.50,2.29
477,119250,41.50,0.87,12.75,4.44,17.75,2.38
478,119500,33.33,6.18,15.50,3.77,17.00,4.32


,Time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23
0,0,42.00,0.00,18.50,0.50,12.00,0.00
1,250,42.00,0.00,18.00,0.00,11.33,0.94
2,500,42.75,0.43,16.75,1.79,18.25,0.43
3,750,42.50,0.50,16.75,0.83,19.00,1.22
4,1000,43.00,0.82,16.25,0.83,18.00,0.00
...,...,...,...,...,...,...,...
475,118750,41.75,1.79,11.00,6.00,16.67,2.49
476,119000,36.33,0.47,16.00,3.16,20.33,1.70
477,119250,31.50,1.50,21.00,0.00,12.25,7.12
478,119500,34.25,6.38,12.67,2.49,15.25,4.21


### (c) Feature Extraction

#### i. Research

Types of time-domain features that are usually used in time series classification:

Mean,median,Variance, Standard Deviation, Skewness, kurtosis and similar

In our case, we are extracing first quartile nd third quartile as well following the HW guidelines.

Citations: http://fastml.com/classifying-time-series-using-feature-extraction/

This article explains the concept of extracing features preally well by giving us a scenerio.


#### ii. Extraction

In [5]:

## create dataframe

final_df = pd.DataFrame()

counter = 0
for roots, dirs, files in sorted(os.walk(dir_path)):
    for each in files:
        print(each)
        if each.endswith('.csv'):
            minimum = []
            maximum = []
            mean = []
            median = []
            standard_deviation = []
            first_quartile = []
            third_quartile = []
            file_path = os.path.join(roots, each)
            counter +=1
            # print(file_path)
            df = pd.read_csv(file_path, skiprows=4)
            ## for each column in df
            for col in df.columns:
                # print(col)
                if col != '# Columns: time':
                    minimum.append(df[col].min())
                    maximum.append(df[col].max())
                    mean.append(df[col].mean())
                    median.append(df[col].median())
                    standard_deviation.append(df[col].std())
                    first_quartile.append(df[col].quantile(0.25))
                    third_quartile.append(df[col].quantile(0.75))

            for i in range(len(minimum)):
                final_df.loc[counter, "Min" + str(i+1)] = minimum[i]
                final_df.loc[counter, "Max" + str(i+1)] = maximum[i]
                final_df.loc[counter, "Mean" + str(i+1)] = mean[i]
                final_df.loc[counter, "Median" + str(i+1)] = median[i]
                final_df.loc[counter, "Std" + str(i+1)] = standard_deviation[i]
                final_df.loc[counter, "1st" + str(i+1)] = first_quartile[i]
                final_df.loc[counter, "3rd" + str(i+1)] = third_quartile[i]


#final_df index as column named Instance
final_df['Instance'] = final_df.index
first_column = final_df.pop('Instance')
final_df.insert(0, 'Instance', first_column)

final_df = pd.DataFrame(final_df)
## transpose the dataframe
# final_df = final_df.T
display(final_df)

        
        

bendingType.pdf
.DS_Store
sensorsPlacement.pdf
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
.DS_Store
dataset2.csv
dataset3.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.csv
dataset8.csv
dataset9.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.csv
dataset8.csv
dataset9.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.csv
dataset8.csv
dataset9.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.

,Instance,Min1,Max1,Mean1,Median1,Std1,1st1,3rd1,Min2,Max2,...,Std5,1st5,3rd5,Min6,Max6,Mean6,Median6,Std6,1st6,3rd6
1,1,36.25,48.00,43.969125,44.500,1.618364,43.31,44.67,0.0,1.50,...,3.318301,20.5000,23.75,0.00,2.96,0.555313,0.490,0.487826,0.0000,0.8300
2,2,37.00,48.00,43.454958,43.250,1.386098,42.50,45.00,0.0,1.58,...,2.488862,22.2500,24.00,0.00,5.26,0.679646,0.500,0.622534,0.4300,0.8700
3,3,33.00,47.75,42.179813,43.500,3.670666,39.15,45.00,0.0,3.00,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.500,0.524317,0.0000,1.0000
4,4,33.00,45.75,41.678063,41.750,2.243490,41.33,42.75,0.0,2.83,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.430,0.389164,0.0000,0.5000
5,5,37.25,45.00,40.624792,40.500,1.476967,39.25,42.00,0.0,1.30,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,84,19.75,45.50,34.322750,35.250,4.752477,31.00,38.00,0.0,13.47,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.200,1.732727,2.1575,4.5650
85,85,19.25,44.00,34.473188,35.000,4.796705,31.25,38.00,0.0,13.86,...,3.156320,13.7300,17.75,0.43,9.00,3.340458,3.090,1.699114,2.1200,4.3750
86,86,23.50,46.25,34.873229,35.250,4.531720,31.75,38.25,0.0,14.82,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.270,1.690960,2.1700,4.5000
87,87,18.33,45.75,34.599875,35.125,4.731790,31.50,38.00,0.0,15.37,...,2.905688,14.0000,18.25,0.00,8.86,3.289542,3.015,1.680170,2.1200,4.2600


#### iii. Standard Deviation

In [6]:
#rid of Instance column

final_df2 = final_df.iloc[:,1:]
sd = final_df2.std(axis = 0) ## vertically
sd = pd.DataFrame(sd)
sd.rename(columns={0: 'Standard Deviation'}, inplace=True)


## copy sd to a new dataframe
display(sd)

,Standard Deviation
Min1,9.569975
Max1,4.394362
Mean1,5.335718
Median1,5.440054
Std1,1.772153
1st1,6.153590
3rd1,5.138925
Min2,0.000000
Max2,5.062729
Mean2,1.574164


In [7]:

## getting rid of first column for final_df
final_df = final_df.iloc[:,1:]
low = []
high = []
for col in final_df.columns:
    confidence_interval = bs.bootstrap(np.array(final_df[col]), stat_func=bs_stats.std, alpha=0.1)
    low.append(confidence_interval.lower_bound)
    high.append(confidence_interval.upper_bound)

## standard Deviation as sd

CITable = pd.DataFrame({'Standard Deviation': sd['Standard Deviation'], 'Lower Bound': low, 'Upper Bound': high})

display(CITable)


,Standard Deviation,Lower Bound,Upper Bound
Min1,9.569975,8.332723,10.819885
Max1,4.394362,3.470936,5.408068
Mean1,5.335718,4.757757,5.929142
Median1,5.440054,4.866050,6.061335
Std1,1.772153,1.584398,1.960895
1st1,6.153590,5.632090,6.694184
3rd1,5.138925,4.401004,5.907230
Min2,0.000000,0.000000,0.000000
Max2,5.062729,4.691013,5.477501
Mean2,1.574164,1.434869,1.744317


#### iv. Select Features

In [8]:
display(final_df)
correlation_matrix = final_df.corr()

# compute the merit score for each feature
merit_scores = []
for column in final_df.columns:
    score = sum(abs(correlation_matrix[column])) / (len(df.columns) - 1)
    merit_scores.append(score)

# rank the features based on their merit scores and select the top three
top_features = sorted(range(len(merit_scores)), key=lambda i: merit_scores[i], reverse=True)[:3]

# print the top three features
for i in top_features:
    print(final_df.columns[i])

,Min1,Max1,Mean1,Median1,Std1,1st1,3rd1,Min2,Max2,Mean2,...,Std5,1st5,3rd5,Min6,Max6,Mean6,Median6,Std6,1st6,3rd6
1,36.25,48.00,43.969125,44.500,1.618364,43.31,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.75,0.00,2.96,0.555313,0.490,0.487826,0.0000,0.8300
2,37.00,48.00,43.454958,43.250,1.386098,42.50,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.00,0.00,5.26,0.679646,0.500,0.622534,0.4300,0.8700
3,33.00,47.75,42.179813,43.500,3.670666,39.15,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.500,0.524317,0.0000,1.0000
4,33.00,45.75,41.678063,41.750,2.243490,41.33,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.430,0.389164,0.0000,0.5000
5,37.25,45.00,40.624792,40.500,1.476967,39.25,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,19.75,45.50,34.322750,35.250,4.752477,31.00,38.00,0.0,13.47,4.456333,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.200,1.732727,2.1575,4.5650
85,19.25,44.00,34.473188,35.000,4.796705,31.25,38.00,0.0,13.86,4.359312,...,3.156320,13.7300,17.75,0.43,9.00,3.340458,3.090,1.699114,2.1200,4.3750
86,23.50,46.25,34.873229,35.250,4.531720,31.75,38.25,0.0,14.82,4.380583,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.270,1.690960,2.1700,4.5000
87,18.33,45.75,34.599875,35.125,4.731790,31.50,38.00,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.00,8.86,3.289542,3.015,1.680170,2.1200,4.2600


Min1
Max1
Mean1


Here I'm using a CFS approach to perform feature selection.

Step 1 :Compute the correlations between each pair of features in the feature table. You can use Pearson correlation coefficient or Spearman's rank correlation coefficient depending on the distribution of your data.

Step 2:Compute the merit score for each feature, which is the average correlation between the feature and all other features.

Step 3:Rank the features based on their merit scores and select the top three features.

Result gives that Min1,Max1 and Mean1 should be selected as the three most important time-domain features from our feature tables.

## 2. ISLR 3.7.4

### (a) Linear Train

I would expect the linear regression RSS to be lower than the nonlinear regression since we assumed a true linear relationship between X and Y. This is because the nonlienar model might overfit to the data.

### (b) Linear Test

Since informations about the test data is unknown to us, we cannot simply to conclude a result. But what we can assume is that nonlinear regresion's RSS will be higher than the linear regression's RSS due to the nonlinear model trained is prone to more error due to overfitting of the dataset.

### (c) Not Linear Train

Since we know that the true relationship between x and Y is not linear, even though we do not know how far it is from the linear relationship, we can safely say that the cubic regression will have lower train RSS than the linear regression's RSS because Cubic regression fits the model better due to higher flexbility. It can display the relationship better by displaying a curve that fits tighter to all the datapoints, thus decreasing the RSS.

### (d) Not Linear Testing

Since we do not know how far it is from the linear this non-linear relationship really is, we cannot conclude that the test RSS will be lower for one and higher for the other. If the true relationship is more linear than cubic, the linear regression test RSS would be lower than cubic regression test RSS and vice versa. This is because of the famous bias-variance tradeoff property.

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 

## 2. Time Series Classification Part 2: Binary and Multiclass Classification

### (a) Binary Classification Using Logistic Regression

##### Assume that you want to use the training set to classify bending from other activities, i.e. you have a binary classification problem. Depict scatter plots of the features you specified in 1(c)iv extracted from time series 1, 2, and 6 of each instance, and use color to distinguish bending vs. other activities. (See p. 129 of the textbook)

In [9]:
dir_path = '../data/AReM/'

train_df = pd.DataFrame()
test_df = pd.DataFrame()

import glob

subdir = os.listdir(dir_path)
subdir = sorted(subdir)

for each in subdir:
    if each == 'bending1':
        for filepath in glob.iglob(dir_path + each + '/*.csv'):

            if 'dataset1' in filepath or 'dataset2' in filepath:
                temp_df = pd.read_csv(filepath, skiprows=4)
                temp_df['class'] = each
                test_df = test_df.append(temp_df)
            else:
                temp_df = pd.read_csv(filepath, skiprows=4)
                temp_df['class'] = each
                train_df = train_df.append(temp_df)
    elif each == 'bending2':
        for filepath in glob.iglob(dir_path + each + '/*.csv'):
            if 'dataset1' in filepath or 'dataset2' in filepath:
                temp_df = pd.read_csv(filepath, skiprows=4)
                temp_df['class'] = each
                test_df = test_df.append(temp_df)
            else:
                temp_df = pd.read_csv(filepath, skiprows=4)
                temp_df['class'] = each
                train_df = train_df.append(temp_df)
    else:
        for filepath in glob.iglob(dir_path + each + '/*.csv'):
            if 'dataset1' in filepath or 'dataset2' in filepath or 'dataset3' in filepath:
                temp_df = pd.read_csv(filepath, skiprows=4)
                temp_df['class'] = each
                test_df = test_df.append(temp_df)
            else:
                temp_df = pd.read_csv(filepath, skiprows=4)
                temp_df['class'] = each
                train_df = train_df.append(temp_df)


train_df.rename(columns={'# Columns: time': 'Time'}, inplace=True)
test_df.rename(columns={'# Columns: time': 'Time'}, inplace=True)



FourAI = pd.DataFrame()
## get column 0 1 2 6 for train_Df
FourAI['1'] = train_df.iloc[:,1]
FourAI['2'] = train_df.iloc[:,2]
FourAI['6'] = train_df.iloc[:,6]
FourAI['class'] = train_df.iloc[:,-1]


## transform the class column based on condition, if bending1 or bending2, then 1, else 0
FourAI['class'] = np.where(FourAI['class'].str.contains('bending1') | FourAI['class'].str.contains('bending2'), 1, 0)

## min, max, mean












display(train_df)
display(test_df)
display(FourAI)


,Time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,class
0,0,42.00,0.00,18.50,0.50,12.00,0.00,bending1
1,250,42.00,0.00,18.00,0.00,11.33,0.94,bending1
2,500,42.75,0.43,16.75,1.79,18.25,0.43,bending1
3,750,42.50,0.50,16.75,0.83,19.00,1.22,bending1
4,1000,43.00,0.82,16.25,0.83,18.00,0.00,bending1
...,...,...,...,...,...,...,...,...
475,118750,31.50,1.66,12.50,3.20,14.25,4.44,walking
476,119000,27.33,1.25,11.33,0.94,20.00,4.00,walking
477,119250,37.80,7.68,14.20,2.48,17.25,0.83,walking
478,119500,33.75,1.30,15.75,5.21,16.50,2.69,walking


,Time,avg_rss12,var_rss12,avg_rss13,var_rss13,avg_rss23,var_rss23,class
0,0,39.25,0.43,22.75,0.43,33.75,1.30,bending1
1,250,39.25,0.43,23.00,0.00,33.00,0.00,bending1
2,500,39.25,0.43,23.25,0.43,33.00,0.00,bending1
3,750,39.50,0.50,23.00,0.71,33.00,0.00,bending1
4,1000,39.50,0.50,24.00,0.00,33.00,0.00,bending1
...,...,...,...,...,...,...,...,...
475,118750,39.33,0.47,10.00,2.16,7.00,4.85,walking
476,119000,39.25,2.49,12.50,1.66,10.75,4.49,walking
477,119250,40.00,8.22,15.75,2.05,14.00,4.42,walking
478,119500,29.50,6.50,18.00,1.87,17.00,3.67,walking


,1,2,6,class
0,42.00,0.00,0.00,1
1,42.00,0.00,0.94,1
2,42.75,0.43,0.43,1
3,42.50,0.50,1.22,1
4,43.00,0.82,0.00,1
...,...,...,...,...
475,31.50,1.66,4.44,0
476,27.33,1.25,4.00,0
477,37.80,7.68,0.83,0
478,33.75,1.30,2.69,0


In [10]:

## create dataframe

final_df = pd.DataFrame()

counter = 0
for roots, dirs, files in sorted(os.walk(dir_path)):
    for each in files:
        print(each)
        if each.endswith('.csv'):
            minimum = []
            maximum = []
            mean = []
            median = []
            standard_deviation = []
            first_quartile = []
            third_quartile = []
            file_path = os.path.join(roots, each)
            counter +=1
            # print(file_path)
            df = pd.read_csv(file_path, skiprows=4)
            ## for each column in df
            for col in df.columns:
                # print(col)
                if col != '# Columns: time':
                    minimum.append(df[col].min())
                    maximum.append(df[col].max())
                    mean.append(df[col].mean())
                    median.append(df[col].median())
                    standard_deviation.append(df[col].std())
                    first_quartile.append(df[col].quantile(0.25))
                    third_quartile.append(df[col].quantile(0.75))

            for i in range(len(minimum)):
                final_df.loc[counter, "Min" + str(i+1)] = minimum[i]
                final_df.loc[counter, "Max" + str(i+1)] = maximum[i]
                final_df.loc[counter, "Mean" + str(i+1)] = mean[i]
                final_df.loc[counter, "Median" + str(i+1)] = median[i]
                final_df.loc[counter, "Std" + str(i+1)] = standard_deviation[i]
                final_df.loc[counter, "1st" + str(i+1)] = first_quartile[i]
                final_df.loc[counter, "3rd" + str(i+1)] = third_quartile[i]


#final_df index as column named Instance
final_df['Instance'] = final_df.index
first_column = final_df.pop('Instance')
final_df.insert(0, 'Instance', first_column)

final_df = pd.DataFrame(final_df)
## transpose the dataframe
# final_df = final_df.T
display(final_df)

        
        

bendingType.pdf
.DS_Store
sensorsPlacement.pdf
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
.DS_Store
dataset2.csv
dataset3.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.csv
dataset8.csv
dataset9.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.csv
dataset8.csv
dataset9.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.csv
dataset8.csv
dataset9.csv
dataset7.csv
dataset6.csv
dataset4.csv
dataset5.csv
dataset1.csv
dataset2.csv
dataset3.csv
dataset10.csv
dataset11.csv
dataset13.csv
dataset12.csv
dataset15.csv
dataset14.

,Instance,Min1,Max1,Mean1,Median1,Std1,1st1,3rd1,Min2,Max2,...,Std5,1st5,3rd5,Min6,Max6,Mean6,Median6,Std6,1st6,3rd6
1,1,36.25,48.00,43.969125,44.500,1.618364,43.31,44.67,0.0,1.50,...,3.318301,20.5000,23.75,0.00,2.96,0.555313,0.490,0.487826,0.0000,0.8300
2,2,37.00,48.00,43.454958,43.250,1.386098,42.50,45.00,0.0,1.58,...,2.488862,22.2500,24.00,0.00,5.26,0.679646,0.500,0.622534,0.4300,0.8700
3,3,33.00,47.75,42.179813,43.500,3.670666,39.15,45.00,0.0,3.00,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.500,0.524317,0.0000,1.0000
4,4,33.00,45.75,41.678063,41.750,2.243490,41.33,42.75,0.0,2.83,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.430,0.389164,0.0000,0.5000
5,5,37.25,45.00,40.624792,40.500,1.476967,39.25,42.00,0.0,1.30,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,84,19.75,45.50,34.322750,35.250,4.752477,31.00,38.00,0.0,13.47,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.200,1.732727,2.1575,4.5650
85,85,19.25,44.00,34.473188,35.000,4.796705,31.25,38.00,0.0,13.86,...,3.156320,13.7300,17.75,0.43,9.00,3.340458,3.090,1.699114,2.1200,4.3750
86,86,23.50,46.25,34.873229,35.250,4.531720,31.75,38.25,0.0,14.82,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.270,1.690960,2.1700,4.5000
87,87,18.33,45.75,34.599875,35.125,4.731790,31.50,38.00,0.0,15.37,...,2.905688,14.0000,18.25,0.00,8.86,3.289542,3.015,1.680170,2.1200,4.2600
